# Facts and variables

When connecting to an host, ansible creates a dictionary with a huge set of useful informations.

This step is named **fact gathering**. 

You can inspect facts using the `setup` module.

In [1]:
cd exercise-03

/notebooks/exercise-03


In [2]:
# Let's run the setup module

!ansible -i inventory -m setup localhost 

 [WARNING]: Host file not found: inventory

 [WARNING]: provided hosts list is empty, only localhost is available

localhost | SUCCESS => {
    "ansible_facts": {
        "ansible_all_ipv4_addresses": [
            "172.17.0.2"
        ], 
        "ansible_all_ipv6_addresses": [
            "fe80::42:acff:fe11:2"
        ], 
        "ansible_apparmor": {
            "status": "disabled"
        }, 
        "ansible_architecture": "x86_64", 
        "ansible_bios_date": "06/30/2014", 
        "ansible_bios_version": "A00", 
        "ansible_cmdline": {
            "BOOT_IMAGE": "/vmlinuz-4.10.17-100.fc24.x86_64", 
            "LANG": "it_IT.UTF-8", 
            "i8042.nopnp": true, 
            "rd.lvm.lv": "vg0/root00", 
            "ro": true, 
            "root": "/dev/mapper/vg0-root00"
        }, 
        "ansible_date_time": {
            "date": "2017-07-01", 
            "day": "01", 
            "epoch": "1498906368", 
            "hour": "10", 
            "iso8601": "2017-07-

## Exercise

 - run the following command
 - what does it do?
 - where's  its output ?
 - use the json module to reindent it

In [3]:
!ansible >/dev/null -i inventory -m setup localhost --tree host_status

 [WARNING]: Host file not found: inventory
 [WARNING]: provided hosts list is empty, only localhost is available


In [4]:
# Solution
!tree host_status
import json
ret = json.load(open('host_status/localhost'))


host_status
└── localhost

0 directories, 1 file


## Exercise

Gather the following infos from the output of the previous command.

  - hostname
  - first ip address
  - distribution
  - kernel release



In [5]:
# use this cell for the exercse

## Exercise

modify [this playbook](/edit/notebooks/exercise-03/get-facts.yml) to print:


  - hostname
  - first ip address
  - distribution
  - kernel release


In [7]:
!ansible-playbook get-facts.yml

 [WARNING]: Host file not found: inventory

 [WARNING]: provided hosts list is empty, only localhost is available


PLAY [Use this section to complete the exercise, and the next section as a reference for the debug module.] ***

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [Dump the requested facts.] ***********************************************
ok: [localhost] => {
    "msg": "A string"
}

PLAY [Those tasks are run on localhost only] ***********************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [This is the debug module.] ***********************************************
ok: [localhost] => {
    "msg": "A string"
}

TASK [The debug module can print variables too] ********************************
ok: [localhost] => {
    "ansible_hostname": "sysadminpy"
}

TASK [We can iterate too] ******************************************************
ok: [localhost] 

## Facts vs Variables

Essentially, facts are per-host variables gathered by ansible.

In the vault lesson we've seen variables in action loaded it via -e @variables.yml.

Now we'll define them with 

```
- hosts: localhost
  vars:
  - one_variable: 1
  - another_variable: 
      that_is: a_map
```

In [10]:
!cat vars-and-facts.yml

---
- hosts: localhost
  vars:
  - one: 1
  - pi: 3.1415
  tasks:
  - debug: 
      msg: >
        {{ansible_hostname }} {{ one }}
  - debug: 
      msg: >
        The output is {{ one + pi }}


- hosts: localhost
  vars:
  - one: 1
  - pi: 3.1415
  tasks:
  - name: Ansible can process expressions using filters
    debug: 
      msg: >
        Floor it  {{ ( one + pi ) | int }}

- hosts: localhost
  name: Use this cell for the exercise
  tasks:
  - name: >
      Show the free percentage of the local disk using
      facts and filters
    debug: msg="Replace me"


In [20]:
# This playbook mixes variables and facts
!ansible-playbook vars-and-facts.yml --tags variables

 [WARNING]: Host file not found: inventory

 [WARNING]: provided hosts list is empty, only localhost is available


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [debug] *******************************************************************
ok: [localhost] => {
    "msg": "sysadminpy 1\n"
}

TASK [debug] *******************************************************************
ok: [localhost] => {
    "msg": "The output is 4.1415\n"
}

PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

PLAY [Use this cell for the exercise] ******************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

PLAY RECAP ***********************************************************

### filters

inside {{ braces }} you can process expressions using filters.

```

  - debug:
      msg: >
        Floor it  {{ ( one + pi ) | int }}


```

In [21]:
# This playbook mixes variables and facts
!ansible-playbook vars-and-facts.yml --tags filters

 [WARNING]: Host file not found: inventory

 [WARNING]: provided hosts list is empty, only localhost is available


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [Ansible can process expressions using filters] ***************************
ok: [localhost] => {
    "msg": "Floor it  4\n"
}

PLAY [Use this cell for the exercise] ******************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=4    changed=0    unreachable=0    failed=0   



## Exercise

Edit [the last section of vars-and-facts.yml](/edit/notebook/exercise-03/vars-and-facts.yml) so that it shows:
    
    - the free percentage of every device

Hints:
    
    - use the playbook as a reference
    - iterate thru server facts

## Conditions and Assertions

Ansible supports basic condition checking via when:
    
```
- hosts: localhost
  tasks:    
  - debug: msg="This always happens"
    when: true
  - debug: msg="This never does"
    when: false
```


You can use `when` with variables and conditions too:

```
- hosts: localhost  
  tasks:    
  - debug: msg="This is {{ansible_kernel}}!"
    when: ansible_system == 'Linux'

```

Remember:

*** when statements should not include jinja2 templating delimiters such as {{ }} or {% %}. ***


In [13]:
!ansible-playbook conditions.yml --tags when

 [WARNING]: Host file not found: inventory

 [WARNING]: provided hosts list is empty, only localhost is available


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [debug] *******************************************************************
ok: [localhost] => {
    "msg": "This always happens"
}

TASK [debug] *******************************************************************
skipping: [localhost]

PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=3    changed=0    unreachable=0    failed=0   



## assertions

The fail module is used to make assertions. Unless told otherwise, ansible playbooks stops when a task fail.

```
- hosts: localhost  
  tags: assert
  tasks:    
  - fail: msg="This is Linux!"
    when: ansible_system != 'Linux'

  - fail: msg="Stop processing now!"
    when: ansible_architecture == 'x86_64'

```

In [18]:
!ansible-playbook conditions.yml --tags assert

 [WARNING]: Host file not found: inventory

 [WARNING]: provided hosts list is empty, only localhost is available


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [debug] *******************************************************************
ok: [localhost] => {
    "msg": "This is Linux!"
}

TASK [fail] ********************************************************************
fatal: [localhost]: FAILED! => {"changed": false, "failed": true, "msg": "Stop processing now!"}

PLAY RECAP *********************************************************************
localhost                  : ok=3    changed=0    unreachable=0    failed=1   



## Exercise

Write a playbook checking that free disk size is > 75%.

  - get informations from gather_facts
  - iterate thru partition `with_items`
  - use `when` and fail module
    